### Retrival Augmentented Generation Day 2

In [ ]:
%pip install langchain_cohere -q
%pip install spacy -q
%pip install psycopg2 -q
%pip install python-dotenv -q
#ignore error

In [ ]:
# now you need to run this in a terminal window
# python -m spacy download en_core_web_md
# now restart your kernel

Standard imports for the libraires we will be using in this notebook.  Try to keep your imports in the first cell so this can this code can more easliy be converted into a python program later

In [1]:
import boto3
import pandas as pd
import json
import time
import os
import numpy as np
import pyarrow
import traceback
from langchain.embeddings import BedrockEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import BedrockChat
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import BedrockEmbeddings
import dbconnection
import psycopg2
from psycopg2 import OperationalError
from dotenv import load_dotenv

load_dotenv()
# Create the AWS client for the Bedrock runtime with boto3
aws_client = boto3.client(service_name="bedrock-runtime")

#### Lets define functions that will use various embedding models so we can generate vector embeddings
Spacey

In [2]:
def generate_spacy_vector_embedding(text):
    embedder = SpacyEmbeddings(model_name="en_core_web_md")
    query_embedding = embedder.embed_query(text)

    return(np.array(query_embedding))

Cohere

In [3]:
# send in an array size of one and only return the 0th element
def generate_cohere_vector_embedding(text_data):
    input_type = "clustering"
    truncate = "NONE" # optional
    model_id = "cohere.embed-english-v3" # or "cohere.embed-multilingual-v3"
    
    # Create the JSON payload for the request
    json_params = {
            'texts': [text_data],
            'truncate': truncate, 
            "input_type": input_type
        }
    json_body = json.dumps(json_params)
    params = {'body': json_body, 'modelId': model_id,}
    
    # Invoke the model and print the response
    result = aws_client.invoke_model(**params)
    response = json.loads(result['body'].read().decode())
    return(np.array(response['embeddings'][0]))


Amazon Titan

In [4]:
# Let's generate a dense vector using Amazon Titan with LangChain
def generate_titan_vector_embedding(text):
    #create an Amazon Titan Text Embeddings client
    embeddings_client = BedrockEmbeddings(region_name="us-west-2") 

    #Invoke the model
    embedding = embeddings_client.embed_query(text)
    return(np.array(embedding))



In [5]:
# Let's generate a dense vector using Amazon Titan without using a np.array as a return value
def generate_vector_embedding(text):
    #create an Amazon Titan Text Embeddings client
    embeddings_client = BedrockEmbeddings(region_name="us-west-2") 

    #Invoke the model
    embedding = embeddings_client.embed_query(text)
    #Note pgvector does not want a np.array as out manual method
    return(embedding)



This is the mathmatical formula to calcuate cosine similarity between 2 vectors

In [6]:
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    similarity = dot_product / (norm_vec1 * norm_vec2)
    return similarity



In [7]:
def clean_value(value):
    value_str = str(value)
    cleaned_value = ''.join(char for char in value_str if char.isalnum() or char.isspace())
    return cleaned_value

In [8]:
def limit_string_size(x, max_chars=2048):
    # Check if the input is a string
    if isinstance(x, str):
        return x[:max_chars]
    else:
        return x

In [9]:
def print_top_values(list_stuff: list, num_items: int) -> None:
    i=0
    for item in list_stuff:
        i=i+1
        if i>num_items:
            return None
        print(item)

In [10]:
# clean abstract text
#df = pd.read_csv('data/latest_research_articles.csv')
#df['abstract'] = df['abstract'].apply(clean_value)

#df
dft = pd.read_pickle('data/embedded_df.pkl')

### Advanced Retrieval Techniques
#### HyDE
A technique that optimizes semantic matching requires better semantic context.  What if we generated a document from the query that better match our stored document?

In [11]:
### Retrieval from embedded sources
#Now that we have a dataframe with embedded content of interest, we can use semantic similarity to retrieve the right data to feed to an LLM

# Given the following query let's generate context that more closely matches the embedded data
query = "What is the latest research for broken ribs in children"

#### Calling the LLM with Python
Before we embed the vector with the query let's transform the query into a fake article.  This article will likely have a larger semantic overlap than the original smaller question. Using Bedrock we will now call Anthropic Claude Sonnet to generate a fictitous article.


In [12]:
# Generate HyDE context

def generate_hyde_response(query_phrase):
    model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
    # Each model will named parameters which will likely be different depending on the providor
    model_kwargs =  { 
        "max_tokens": 400, # This is the maximum output tokens you want the model to use
        "temperature": 1,  # Temperature controls the randomness and creativity of the generated text.
        "top_k": 250,      # Top-k parameter determines the number of highest probability next word choices the model should conside
        "top_p": 0.9,      # Top-p sampling considers the cumulative probability distribution of the next word choices and sets a probability threshold
        "stop_sequences": ["\n\nHuman"],
    }
    # LangChain tooling
    model = BedrockChat(
        client=aws_client,
        model_id=model_id,
        model_kwargs=model_kwargs,
    )
    
    human_prompt = "Given the following question \n {query} can you please generate a paragraph of text that answers the question. Be sure to use scientific \
                    medical terminology. Please just include the paragraph in your response."
    # Uses the messaging method which is required for all Claude 3 calls
    messages = [
        ("system", "You are a helpful assistant"),
        ("human", human_prompt),
    ]
    try:
        prompt = ChatPromptTemplate.from_messages(messages)
        # LangChain at work
        chain = prompt | model | StrOutputParser()


        # Send the message content to Claude using Bedrock and get the response
        start_time = time.time()  # Start timing
        # Call Bedrock
        response = chain.invoke({"query": query_phrase})
        end_time = time.time()  # End timing
        print("Claude call took :", end_time - start_time)  # Calculate execution time

        return(response)
    except Exception as e:
        exc_type, exc_value, exc_traceback = traceback.sys.exc_info()
        line_number = exc_traceback.tb_lineno
        print(f"Errort: {exc_type}{exc_value}{exc_traceback} on {line_number}")

In [13]:
print(generate_hyde_response(query))

Claude call took : 3.290161609649658
The management of rib fractures in pediatric patients has undergone significant advancements in recent years. Clinicians now employ a multidisciplinary approach, combining pharmacological interventions with innovative therapeutic modalities. Analgesic regimens, encompassing opioid and non-opioid medications, are tailored to alleviate acute pain and facilitate respiratory function. Concurrently, emerging techniques like ultrasound-guided regional anesthesia and cryotherapy have demonstrated promising results in pain management and reducing the risk of complications. Furthermore, researchers are exploring the potential benefits of early mobilization and targeted physiotherapy protocols, aimed at enhancing recovery and minimizing long-term respiratory impairments.


#### Titan Embeddings - SAME No HyDE

In [14]:
# Let's search our records for a good semantic search
query_vector = generate_titan_vector_embedding(query)

results = []
# Iterate over each row in the DataFrame
for index, row in dft.iterrows():
    # Extract the value from the specified column
    article_embedding = row['embedded_abstract']
    results.append((index, cosine_similarity(article_embedding, query_vector)))
    #print (index, value)

results.sort(key=lambda x: x[1], reverse=True)
i = 0
# Print the sorted data
print("Here are a few articles that may match your interest:")
for item in results:
    article_title = dft.iloc[item[0]]['title']
    print(f"Abtract: '{article_title}' with a cosine match of: {item[1]}")
    i=i+1
    if i == 5:
        break

Here are a few articles that may match your interest:
Abtract: 'High sensitivity methods for automated rib fracture detection in pediatric radiographs' with a cosine match of: 0.46759500523692665
Abtract: 'Magnetic resonance imaging based finite element modelling of the proximal femur: a short-term in vivo precision study' with a cosine match of: 0.23057253235100217
Abtract: 'On the crashworthiness analysis of bio-inspired DNA tubes' with a cosine match of: 0.21674978237483608
Abtract: 'Reproduction of forearm rotation dynamic using intensity-based biplane 2D–3D registration matching method' with a cosine match of: 0.19907903320363604
Abtract: 'Propagation of extended fractures by local nucleation and rapid transverse expansion of crack-front distortion' with a cosine match of: 0.19056102046718387


Now let's compare our cosine scores with HyDE . . .

In [15]:

# Let's search our records for a good semantic search
query_vector = generate_titan_vector_embedding(generate_hyde_response(query))
# This is a tuple of the article index and the cosine similarity score
results = []
# Iterate over each row in the DataFrame
for index, row in dft.iterrows():
    # Extract the value from the specified column
    article_embedding = row['embedded_abstract']
    results.append((index, cosine_similarity(article_embedding, query_vector)))
    #print (index, value)

results.sort(key=lambda x: x[1], reverse=True)
i = 0
# Print the sorted data
print("Here are a few articles that may match your interest:")
for item in results:
    # Use the index from the Original dataframe to extract values of interest
    article_title = dft.iloc[item[0]]['title']
    print(f"Abtract: '{article_title}' with a cosine match of: {item[1]}")
    i=i+1
    if i == 5:
        break

Claude call took : 6.0228471755981445
Here are a few articles that may match your interest:
Abtract: 'High sensitivity methods for automated rib fracture detection in pediatric radiographs' with a cosine match of: 0.6292173704538236
Abtract: 'AI co-pilot bronchoscope robot' with a cosine match of: 0.32338228689168846
Abtract: 'Non-invasive biomarkers for detecting progression toward hypovolemic cardiovascular instability in a lower body negative pressure model' with a cosine match of: 0.31326814424948946
Abtract: 'Magnetic resonance imaging based finite element modelling of the proximal femur: a short-term in vivo precision study' with a cosine match of: 0.26150022736172723
Abtract: 'Development and validation of a semi-automated and unsupervised method for femur segmentation from CT' with a cosine match of: 0.2510235206731928


#### Differentiating the retrievals and ranking
 How can we evaluate if the cosine similarities are different enough?  Thresholds aren't consistent enough.  What about Z-Score?

In [16]:
def calculate_zscores(cosine_scores):
    zscores = []
    # Calculate the mean of the sample points
    mean = np.mean(cosine_scores)
    # Calculate the standard deviation of the sample points
    std_deviation = np.std(cosine_scores, ddof=1)  # ddof=1 for sample standard deviation
    # Calculate the z-scores for each sample point
    z_scores = [(x - mean) / std_deviation for x in cosine_scores]

    return z_scores

Let's review our distribution of cosine scores and evaluate a threshold for selection.  We will review z-score and see how that compares.

In [17]:
# grab the cosine_scores from the results tuple
cosine_scores = [item[1] for item in results]
print("The top few cosine-scores")
print_top_values(cosine_scores,5)
z_scores = calculate_zscores(cosine_scores)
# Let's print the top 5 z-scores
print("The top few Z-scores")
print_top_values(z_scores, 5)



The top few cosine-scores
0.6292173704538236
0.32338228689168846
0.31326814424948946
0.26150022736172723
0.2510235206731928
The top few Z-scores
12.467139857935173
5.885293812988393
5.667628367375577
4.553536239506188
4.32806808955148


#### Sample values for cosine similarity I sampled for another project I used RAG on

<img src="zscore-dist.jpg" alt="Sample Z-Score Distributions" width="300" height="600">

In [18]:
# Using our new technique we will concat all relvant results and prepare to send that as context to our LLM
articles = ""
i=0
j=0
first_z_score = z_scores[0]
for item in results:
    # Use the index from the Original dataframe to extract values of interest
    article_title = dft.iloc[item[0]]['title']
    abstract = dft.iloc[item[0]]['abstract']
    # If the highest Z score is 2x this record then likey not as significant
    if(first_z_score/2)<z_scores[i]:
        print(f"Abstract: '{article_title}' with a cosine match of: {item[1]} and Z-score of: {z_scores[i]}")
        articles = articles + " " + abstract
        j=j+1
    i=i+1
print(f"Grabbed the top {j} scores")    

Abstract: 'High sensitivity methods for automated rib fracture detection in pediatric radiographs' with a cosine match of: 0.6292173704538236 and Z-score of: 12.467139857935173
Grabbed the top 1 scores


#### Generation
Now lets define a function that will call the LLM with the right prompt to generate a summary of the retrieved context

In [19]:
# Now let's take the records that are greater that 1/2 the top Z-score and send those to the LLM for an answer
def best_answer(data, question):
    model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

    model_kwargs =  { 
        "max_tokens": 2048,
        "temperature": 0.0,
        "top_k": 250,
        "top_p": 0.9,
        "stop_sequences": ["\n\nHuman"],
    }

    model = BedrockChat(
        client=aws_client,
        model_id=model_id,
        model_kwargs=model_kwargs,
    )

    human_prompt = "You are to answer the question using the data in the following information.  Do not make up your answer, only use \
                    supporting data from the article, If you don't have enough data simply respond, I don't have enough information to answer that question. \
                    given the following article data {data} can you please give a concise answer to the following question. {question}"
    messages = [
        ("system", "You are a helpful assistant that can answer quesitons based on news articles you have been given."),
        ("human", human_prompt),
    ]
    try:
        prompt = ChatPromptTemplate.from_messages(messages)

        chain = prompt | model | StrOutputParser()

        # Chain Invoke
        
    
        # Send the message content to Claude using Bedrock and get the response
        start_time = time.time()  # Start timing
        # Call Bedrock
        response = chain.invoke({"data": data,"question": question})
        end_time = time.time()  # End timing
        #print("Claude call took :", end_time - start_time)  # Calculate execution time

        return(response)
    except Exception as e:
        exc_type, exc_value, exc_traceback = traceback.sys.exc_info()
        line_number = exc_traceback.tb_lineno

        return f"ERROR generating good answer: {exc_type}{exc_value}{exc_traceback} on {line_number}"


In [20]:
print("Here is the best answer I could find about ", query)
print(best_answer(articles, query))

Here is the best answer I could find about  What is the latest research for broken ribs in children
Based on the information provided in the article, the latest research focuses on improving the detection and localization of rib fractures in pediatric chest radiographs using computer vision and machine learning techniques. Specifically:

- The research implemented convolutional neural network (CNN) architectures like RetinaNet and YOLOv5, along with an "avalanche decision" scheme to dynamically adjust detection thresholds, to increase the sensitivity (recall) in detecting rib fractures.

- Multiple image preprocessing techniques and model ensembling were used to further enhance detection performance.

- Using a dataset of 1,109 pediatric chest radiographs manually labeled by radiologists, the best performing model achieved an F2 score of 0.725, approaching the expert inter-reader performance of 0.732.

- The goal was to develop methods that can identify all rib fractures to aid radiolo

#### RAG is rad
Let's explore the power of RAG with another example.  Using a larger document with chunking

In [21]:
# Load your large text document
with open("data/staff-report.txt", "r") as file:
    text = file.read()

# Create a text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,  # Adjust the chunk size as needed
    chunk_overlap=512,  # Adjust the overlap between chunks as needed
)

# Split the text into chunks
chunks = text_splitter.split_text(text)

# create a dataframe with new chunk raw text
chunk_df = pd.DataFrame({'raw_text': chunks})

chunk_df['text_embed'] = chunk_df['raw_text'].apply(generate_titan_vector_embedding)


In [22]:
# Let's search our records for a good semantic search
#query = "What is the city replacing at the water treatment plant and why?"
query = "What is the city recommending?"
#query = "What will it cost to replace the mixer gearbox?"

query_vector = generate_titan_vector_embedding(query)
# This is a tuple of the article index and the cosine similarity score
results = []
# Iterate over each row in the DataFrame
for index, row in chunk_df.iterrows():
    # Extract the value from the specified column
    article_embedding = row['text_embed']
    results.append((index, cosine_similarity(article_embedding, query_vector)))
    #print (index, cosine_similarity(article_embedding, query_vector))

results.sort(key=lambda x: x[1], reverse=True)
cosine_scores = [item[1] for item in results]
z_scores = calculate_zscores(cosine_scores)


print("The top few cosine-scores")
print_top_values(cosine_scores,5)
# Let's print the top 5 z-scores
print("The top few Z-scores")
print_top_values(z_scores, 5)

#save the articles that were a good match
articles = ""
i=0
j=0
first_z_score = z_scores[0]
for item in results:
    # Use the index from the Original dataframe to extract values of interest
    chunk_txt = chunk_df.iloc[item[0]]['raw_text']
    # If the highest Z score is 2x this record then likey not as significant
    if(first_z_score/2)<z_scores[i]:
        print(f"Using chunk with a cosine match of: {item[1]} and Z-score of: {z_scores[i]}")
        articles = articles + " " + chunk_txt
        j=j+1
    i=i+1
print(f"Grabbed the top {j} scores")  
print("Here is the best answer I could find about ", query)
print(best_answer(articles, query))
  

The top few cosine-scores
0.37564064545805465
0.3544211509728883
0.32852842656319053
0.32438156860757617
0.31285842398605307
The top few Z-scores
1.731866991010253
1.3917218260436985
0.9766655191214498
0.9101920371828787
0.7254778381259019
Using chunk with a cosine match of: 0.37564064545805465 and Z-score of: 1.731866991010253
Using chunk with a cosine match of: 0.3544211509728883 and Z-score of: 1.3917218260436985
Using chunk with a cosine match of: 0.32852842656319053 and Z-score of: 0.9766655191214498
Using chunk with a cosine match of: 0.32438156860757617 and Z-score of: 0.9101920371828787
Grabbed the top 4 scores
Here is the best answer I could find about  What is the city recommending?
Based on the information provided, the city is recommending to authorize the replacement of ActiFlo mixer gearboxes as part of the Water Treatment Plant major facility maintenance. The recommendation states that this item qualifies as a sole source purchase under criteria C, as Philadelphia Mixing

### Vector Database for Larger Datasets
For our first example we used a local dataframe to store the contents of our raw text and embeddings, then manually calculated similarity between embeddings.  There are other tools that are better suited for larger datasets with embeddings. They are generally called vector databases.  We will explore pgvector running on a Postgresql database engine.

In [23]:
from requests import get

ip = get('https://api.ipify.org').content.decode('utf8')
print('My public IP address is: {}'.format(ip))


My public IP address is: 52.27.224.182


In [24]:
# Fill in your username so yoru database entry will be unique to you
#MY_USERNAME = "dkraker@calpoly.edu"
MY_USERNAME = "YOUR USERNAME HERE"

Let's define a function that will put data into our vector DB

In [25]:
def insert_record_into_db(table_name, record, conn):

    # Dynamically generates and executes an INSERT SQL statement for PostgreSQL, handling
    # special data types like datetime objects and arrays directly.
    
    # Args:
    # - table_name (str): The name of the table into which the record will be inserted.
    # - data (dict): A dictionary representing the record to be inserted, where keys are column names
    #                  and values can include native PostgreSQL types like datetime and arrays.
    # - conn (psycopg2.connection): A psycopg2 connection object.
    
    # Generate column names and placeholders
    columns = ', '.join(record.keys())
    placeholders = ', '.join(['%s'] * len(record))  # PostgreSQL uses %s as placeholder

    # Create the INSERT INTO statement
    sql = f'INSERT INTO {table_name} ({columns}) VALUES ({placeholders})'
    #print("columns=", columns)
    #print("placeholders=", placeholders)
    #print(sql)
    try:
        cur = conn.cursor()
        
        cur.execute(sql, tuple(record.values()))
        conn.commit()
        print("Record inserted successfully.")
    except psycopg2.Error as e:
        print("An error occurred:", e)


In [28]:
# Load your large text document
with open("data/staff-report.txt", "r") as file:
    text = file.read()

# Create a text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,  # Adjust the chunk size as needed
    chunk_overlap=512,  # Adjust the overlap between chunks as needed
)

# Split the text into chunks
chunks = text_splitter.split_text(text)

conn = dbconnection.open_connection_to_db()
try:
    # create a dataframe with new chunk raw text
    for chunk in chunks:
        data_record = {}
        v_embed = generate_vector_embedding(chunk)
        data_record["username"] = MY_USERNAME
        data_record["textattribute1"] = chunk
        data_record["textattribute2"] = ""
        data_record["textattribute3"] = ""
        data_record["textattribute4"] = ""
        data_record["textattribute5"] = ""
        data_record["textembedding1"] = v_embed
        insert_record_into_db("rag", data_record, conn)        
        
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    if conn:
        conn.close()

Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.


Now let's use the database as a way to find the best match. Notice the 2 different commented SQL syntax differences.  We can easily compute Euclidean distance as well

In [31]:
def run_similarity_search_pgvector(question, embedded_text, conn):
    # Cosine similarity
    #1-(textembedding1 <=> ('{embedded_text}')) as cosine_similar  \
    # Euclidean distance
    #textembedding1 <-> ('{embedded_text}') as euclidean_distance  \
    sql = f"SELECT textattribute1, textattribute2, textattribute3, textattribute4, textattribute5, \
                1-(textembedding1 <=> ('{embedded_text}')) as cosine_similar  \
                FROM public.rag \
                WHERE username = '{MY_USERNAME}' \
                ORDER BY cosine_similar DESC \
                LIMIT 50"
    #print(sql)
    cosine_scores = []
    try:
        article_text = ""
        cur = conn.cursor()
        cur.execute(sql)
        rows = cur.fetchall()

        # grab the cosine scores so we can compute Z score for narrow article selection
        # need all scores so we can calc Z
        for row in rows:
            #print(row[5])
            cosine_scores.append(row[5])
            
        z_scores = calculate_zscores(cosine_scores)
        answer = "Unknown"
        article_text = ""
        zscore_index = 0
        first_z_score = z_scores[0]
        for row in rows:
            if(first_z_score/2)<z_scores[zscore_index]:
                print(f"Using chunk with a cosine match of: {row[5]} and Z-score of: {z_scores[zscore_index]}")
                article_text = article_text + row[0] + "\n"
            zscore_index += 1
        #print(article_text)
        answer = best_answer(article_text, question)
        #Close cursor and connection
        cur.close()
        return answer
        
    except psycopg2.Error as e:
            print("An error occurred:", e)
    finally:
        if conn:
            conn.close()
        
  

In [32]:
#query = "What is the city recommending?"
query = "What will it cost to replace the mixer gearbox?"

query_vector = generate_vector_embedding(query)

conn = dbconnection.open_connection_to_db()

run_similarity_search_pgvector(query, query_vector, conn)

Using chunk with a cosine match of: 19.49773609028711 and Z-score of: 1.20770147938811
Using chunk with a cosine match of: 19.398213318025984 and Z-score of: 1.1365966074246054
Using chunk with a cosine match of: 19.363000316324946 and Z-score of: 1.1114383856376002
Using chunk with a cosine match of: 19.06951395040491 and Z-score of: 0.9017546117870814
Using chunk with a cosine match of: 18.780663403380984 and Z-score of: 0.6953829372839252
Using chunk with a cosine match of: 18.658562853913413 and Z-score of: 0.6081471847087676


"According to the information provided, the total estimated cost to replace the mixer gearboxes for the Actiflo ballasted flocculation system at the City's Water Treatment Plant is $500,322. Specifically, it states that the bid estimate from the sole source vendor MISCOwater for the replacement parts is $416,935, and with a 20% contingency added, the total project cost would be $500,322."

In [ ]:
def view_my_data(conn):
    try:
        cur = conn.cursor()

        # SQL statement to delete rows where username is 'bob'
        sql = f"SELECT textattribute1, textattribute2, textattribute3, textattribute4, textattribute5 FROM public.rag WHERE username = '{MY_USERNAME}'"
    
        # Execute the SQL statement
        cur.execute(sql)
        rows = cur.fetchall()

        # grab the cosine scores so we can compute Z score for narrow article selection
        for row in rows:
            print(row[0], row[1], row[2], row[3], row[4])
    
        

        
    except psycopg2.Error as e:
            print("An error occurred:", e)
    finally:
        if conn:
            conn.close()
    

In [ ]:
conn = dbconnection.open_connection_to_db()
view_my_data(conn)

In [26]:
def purge_my_data(conn):
    
    try:
        cur = conn.cursor()

        # SQL statement to delete rows where username is 'bob'
        sql = f"DELETE FROM public.rag WHERE username = '{MY_USERNAME}'"
    
        # Execute the SQL statement
        cur.execute(sql)
    
        # Commit the changes to the database
        conn.commit()
    
        # Get the number of affected rows
        deleted_rows = cur.rowcount
        print(f"{deleted_rows} row(s) deleted.")

        
    except psycopg2.Error as e:
            print("An error occurred:", e)
    finally:
        if conn:
            conn.close()

In [27]:
conn = dbconnection.open_connection_to_db()
purge_my_data(conn)

17 row(s) deleted.


In [ ]:
conn = dbconnection.open_connection_to_db()
try:
    data_record = {}
    data_record["username"] = MY_USERNAME
    data_record["textattribute1"] = "TEST1"
    data_record["textattribute2"] = "TEST2"
    data_record["textattribute3"] = "TEST3"
    data_record["textattribute4"] = "TEST4"
    data_record["textattribute5"] = "TEST5"
    data_record["textembedding1"] = generate_vector_embedding("TEST1")
    data_record["textembedding2"] = generate_vector_embedding("TEST2")
    data_record["textembedding3"] = generate_vector_embedding("TEST3")
    data_record["textembedding4"] = generate_vector_embedding("TEST4")
    
    insert_record_into_db("rag", data_record, conn)              
    #print(data_record)

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    if conn:
        conn.close()